# 📌 Pipeline Trích Xuất Đặc Trưng Hu’s Moments

## **1. Chuẩn bị ảnh**
- Đọc ảnh gốc.
- Chuyển ảnh sang **grayscale**.
- Giảm nhiễu nếu cần (Gaussian blur, median filter).
- Chuyển ảnh grayscale thành ảnh nhị phân (binary) bằng **thresholding**:
  - `cv2.threshold` (global threshold).
  - Hoặc **Otsu’s threshold** / **adaptive threshold**.
- Tìm contour chính bằng `cv2.findContours`.
- Chọn **contour lớn nhất** (thường là đối tượng chính).

> **Mục đích:** Chuẩn hoá ảnh, tách đối tượng chính ra khỏi background, và chọn vùng dữ liệu để tính moments.

---

## **2. Tính diện tích và trọng tâm**
- **Spatial moments**:
$$
m_{pq} = \sum_{x} \sum_{y} x^{p} y^{q} I(x, y)
$$

- **Diện tích (m00)**:
$$
m_{00} = \sum_{x} \sum_{y} I(x, y)
$$

- **Trọng tâm (centroid)**:
$$
\bar{x} = \frac{m_{10}}{m_{00}}, \quad
\bar{y} = \frac{m_{01}}{m_{00}}
$$

> **Mục đích:** Xác định vị trí trung tâm và kích thước của đối tượng.

---

## **3. Tính Central Moments**
$$
\mu_{pq} = \sum_{x} \sum_{y} (x - \bar{x})^{p} (y - \bar{y})^{q} I(x, y)
$$

> **Mục đích:** Loại bỏ ảnh hưởng của vị trí (translation invariance).

---

## **4. Tính Normalized Central Moments**
- Công thức chuẩn:
$$
\eta_{pq} = \frac{\mu_{pq}}{\mu_{00}^{1 + \frac{p+q}{2}}}
$$

- Nếu dùng công thức trong tài liệu bạn học:
$$
\eta_{pq} = \frac{\mu_{pq}}{\mu_{00}^2}
$$

> **Mục đích:** Loại bỏ ảnh hưởng của tỉ lệ (scale invariance).

---

## **5. Tính Hu Moments**
Áp dụng công thức invariants của Hu Moments:

\[
\begin{aligned}
\phi_1 &= \eta_{20} + \eta_{02}, \\
\phi_2 &= (\eta_{20} - \eta_{02})^2 + 4\eta_{11}^2, \\
\phi_3 &= (\eta_{30} - 3\eta_{12})^2 + (3\eta_{21} - \eta_{03})^2, \\
\phi_4 &= (\eta_{30} + \eta_{12})^2 + (\eta_{21} + \eta_{03})^2, \\
\phi_5 &= (\eta_{30} - 3\eta_{12})(\eta_{30} + \eta_{12})[(\eta_{30} + \eta_{12})^2 - 3(\eta_{21} + \eta_{03})^2] \\
&+ (3\eta_{21} - \eta_{03})(\eta_{21} + \eta_{03})[3(\eta_{30} + \eta_{12})^2 - (\eta_{21} + \eta_{03})^2], \\
\phi_6 &= (\eta_{20} - \eta_{02})[(\eta_{30} + \eta_{12})^2 - (\eta_{21} + \eta_{03})^2] + 4\eta_{11}(\eta_{30} + \eta_{12})(\eta_{21} + \eta_{03}), \\
\phi_7 &= (3\eta_{21} - \eta_{03})(\eta_{30} + \eta_{12})[(\eta_{30} + \eta_{12})^2 - 3(\eta_{21} + \eta_{03})^2] \\
&- (\eta_{30} - 3\eta_{12})(\eta_{21} + \eta_{03})[3(\eta_{30} + \eta_{12})^2 - (\eta_{21} + \eta_{03})^2].
\end{aligned}
\]





In [47]:
import os
import cv2
import numpy as np

def convert_to_humoment_folder(input_root, output_root):
    """
    Duyệt toàn bộ ảnh trong input_root (kể cả thư mục con).
    - Chuyển sang ảnh 'vật thể trắng - nền đen' (binary mask).
    - Ghi đè trực tiếp trong output_root (không tạo thêm thư mục con).
    """
    os.makedirs(output_root, exist_ok=True)

    for root, dirs, files in os.walk(input_root):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                input_path = os.path.join(root, file)

                # Lưu kết quả trực tiếp trong output_root
                output_path = os.path.join(output_root, file)

                # Đọc ảnh
                img = cv2.imread(input_path)
                if img is None:
                    print(f"❌ Không đọc được ảnh: {input_path}")
                    continue

                # Chuyển sang ảnh xám
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Làm mịn ảnh để loại bỏ nhiễu
                blurred = cv2.GaussianBlur(gray, (5, 5), 0)

                # Dùng Otsu threshold + THRESH_BINARY_INV để vật thể trắng, nền đen
                _, binary_mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Lưu ảnh nhị phân kết quả
                cv2.imwrite(output_path, binary_mask)
                print(f"✅ Đã xử lý: {output_path}")

    print(f"\n🎯 Xử lý hoàn tất. Kết quả lưu tại: {output_root}")


# Ví dụ chạy
folder_input = r"/media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs"
folder_binary = r"/media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary"

convert_to_humoment_folder(folder_input, folder_binary)


✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_1.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_10.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_2.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_3.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_4.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_5.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_6.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_7.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_8.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary/La_Bang_9.jpg
✅ Đã xử lý: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/

In [48]:
import os
import cv2
import numpy as np
import pandas as pd

def hu_moments_manual(binary_img):
    """
    Tính Hu Moments thủ công từ ảnh nhị phân (0-background, 1-foreground).
    """
    m00 = np.sum(binary_img)
    if m00 == 0:
        return np.zeros(7)

    m10 = np.sum(np.arange(binary_img.shape[1])[None, :] * binary_img)
    m01 = np.sum(np.arange(binary_img.shape[0])[:, None] * binary_img)

    x_bar = m10 / m00
    y_bar = m01 / m00

    mu = {}
    rows, cols = binary_img.shape
    x_indices = np.arange(cols)
    y_indices = np.arange(rows)

    for p in range(4):
        for q in range(4):
            if p + q <= 3:
                term = ((x_indices[None, :] - x_bar) ** p) * ((y_indices[:, None] - y_bar) ** q) * binary_img
                mu[(p, q)] = np.sum(term)

    eta = {}
    for (p, q), val in mu.items():
        eta[(p, q)] = val / (mu[(0, 0)] ** (1 + (p + q) / 2))

    phi = np.zeros(7)
    phi[0] = eta[(2, 0)] + eta[(0, 2)]
    phi[1] = (eta[(2, 0)] - eta[(0, 2)]) ** 2 + 4 * eta[(1, 1)] ** 2
    phi[2] = (eta[(3, 0)] - 3 * eta[(1, 2)]) ** 2 + (3 * eta[(2, 1)] - eta[(0, 3)]) ** 2
    phi[3] = (eta[(3, 0)] + eta[(1, 2)]) ** 2 + (eta[(2, 1)] + eta[(0, 3)]) ** 2
    phi[4] = (eta[(3, 0)] - 3 * eta[(1, 2)]) * (eta[(3, 0)] + eta[(1, 2)]) * \
             ((eta[(3, 0)] + eta[(1, 2)]) ** 2 - 3 * (eta[(2, 1)] + eta[(0, 3)]) ** 2) + \
             (3 * eta[(2, 1)] - eta[(0, 3)]) * (eta[(2, 1)] + eta[(0, 3)]) * \
             (3 * (eta[(3, 0)] + eta[(1, 2)]) ** 2 - (eta[(2, 1)] + eta[(0, 3)]) ** 2)

    phi[5] = (eta[(2, 0)] - eta[(0, 2)]) * \
             ((eta[(3, 0)] + eta[(1, 2)]) ** 2 - (eta[(2, 1)] + eta[(0, 3)]) ** 2) + \
             4 * eta[(1, 1)] * (eta[(3, 0)] + eta[(1, 2)]) * (eta[(2, 1)] + eta[(0, 3)])

    phi[6] = (3 * eta[(2, 1)] - eta[(0, 3)]) * (eta[(3, 0)] + eta[(1, 2)]) * \
             ((eta[(3, 0)] + eta[(1, 2)]) ** 2 - 3 * (eta[(2, 1)] + eta[(0, 3)]) ** 2) - \
             (eta[(3, 0)] - 3 * eta[(1, 2)]) * (eta[(2, 1)] + eta[(0, 3)]) * \
             (3 * (eta[(3, 0)] + eta[(1, 2)]) ** 2 - (eta[(2, 1)] + eta[(0, 3)]) ** 2)

    return phi


def compute_hu_for_folder(binary_folder, csv_path):
    """
    Tính Hu Moments cho tất cả ảnh trong binary_folder (kể cả subfolder) và lưu ra CSV.
    """
    results = []
    for root, dirs, files in os.walk(binary_folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
                path = os.path.join(root, file)
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                binary_img = (img // 255).astype(np.uint8)
                hu_features = hu_moments_manual(binary_img)
                results.append([path] + hu_features.tolist())

    # Chuyển sang DataFrame và lưu CSV
    cols = ["path"] + [f"hu{i+1}" for i in range(7)]
    df = pd.DataFrame(results, columns=cols)
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ Đã lưu Hu Moments ra CSV: {csv_path}")
    return df


# Ví dụ chạy:
folder_binary = r"/media/pphong/New Volume/ML & DL/ML/Hu'smoment/dataset_leafs_binary"
csv_output = r"/media/pphong/New Volume/ML & DL/ML/Hu'smoment/hu_moments.csv"

df_hu = compute_hu_for_folder(folder_binary, csv_output)

# Xem 5 dòng đầu tiên
print(df_hu.head())


✅ Đã lưu Hu Moments ra CSV: /media/pphong/New Volume/ML & DL/ML/Hu'smoment/hu_moments.csv
                                                path       hu1       hu2  \
0  /media/pphong/New Volume/ML & DL/ML/Hu'smoment...  0.299300  0.057548   
1  /media/pphong/New Volume/ML & DL/ML/Hu'smoment...  0.317121  0.067825   
2  /media/pphong/New Volume/ML & DL/ML/Hu'smoment...  0.338576  0.079405   
3  /media/pphong/New Volume/ML & DL/ML/Hu'smoment...  0.367845  0.100857   
4  /media/pphong/New Volume/ML & DL/ML/Hu'smoment...  0.314911  0.066690   

        hu3       hu4       hu5       hu6       hu7  
0  0.009038  0.005692  0.000041  0.001347 -0.000002  
1  0.011022  0.007326  0.000066  0.001883 -0.000002  
2  0.012056  0.008320  0.000083  0.002333  0.000001  
3  0.018173  0.012059  0.000178  0.003645  0.000011  
4  0.010659  0.007069  0.000061  0.001801 -0.000002  
